Import des bibliothèques 

In [49]:
import numpy as np
import os
from PIL import Image 
import tensorflow as tf
import tensorflow_datasets as tfds
import pandas as pd
import matplotlib.pyplot as plt

Chargement des données

In [50]:

csv_train = pd.read_csv('boneage_training_dataset_300.csv') 
csv_test = pd.read_csv('boneage_test_dataset_100.csv')

# Conversion en array du csv pour manipulation plus facile 

### Pour la base d'apprentissage
train_array = csv_train.to_numpy()

numero_patient_train = train_array[:,0]
age_osseux_train = train_array[:,1]
sexe_train = train_array[:,2]

### Pour la base de test 
test_array = csv_test.to_numpy()
numero_patient_test = test_array[:,0]
sexe_test = test_array[:,1]


In [53]:
def chargement_images_train(numero_patient_train):
    img_path = f'training/{numero_patient_train}.png'
    img = tf.io.read_file(img_path)
    img = tf.image.decode_image(img, channels=3)

    return img

def chargement_images_test(numero_patient_train):
    img_path = f'test/{numero_patient_train}.png'
    img = tf.io.read_file(img_path)
    img = tf.image.decode_image(img, channels=3)

    return img

#affichage de l'image 
#image_1600 = chargement_images_train(1600)
#plt.imshow(image_1600.numpy().astype("uint8"))